In [7]:
from kiteconnect import KiteConnect
from datetime import datetime
import sqlite3
import time
import time
from datetime import datetime, time as dt_time
from concurrent.futures import ThreadPoolExecutor
from defs import net_quant_zero,get_symbol_lotsize,place_order,get_expiry_date_and_strike_from_instrument_token,get_name_from_instrument_token,get_instru_tradesymbol_pe_from_ce,cal_dates,short_straddle
import pandas as pd
login = pd.read_excel('login.xlsx')

In [ ]:
# Checking connection
try:
    sqliteConnection = sqlite3.connect('SQLite_Python.db')
    cursor = sqliteConnection.cursor()
    # print("Database created and Successfully Connected to SQLite")

    sqlite_select_Query = "select sqlite_version();"
    cursor.execute(sqlite_select_Query)
    record = cursor.fetchall()
    # print("SQLite Database Version is: ", record)
    cursor.close()

except sqlite3.Error as error:
    print("Error while connecting to sqlite", error)
finally:
    if sqliteConnection:
        sqliteConnection.close()
        # print("The SQLite connection is closed")

In [ ]:
# Creating table

try:
    # Connect to the SQLite database or create it if not exists
    sqliteConnection = sqlite3.connect('SQLite_Python.db')

    # Create a cursor to interact with the database
    cursor = sqliteConnection.cursor()
    # print("Database created and Successfully Connected to SQLite")

    # 1. Create a table named 'portfolio' with columns: name, lot_size, atm, timestamp
    create_table_query = '''
        CREATE TABLE IF NOT EXISTS portfolio (
            tradingsymbol TEXT,
            quantity INTEGER,
            instrument_token TEXT,
            sell_price INTEGER,
            timestamp DATETIME
        );
    '''
    cursor.execute(create_table_query)
    # print("Table 'portfolio' created successfully")
    # Close the cursor
    cursor.close()
except sqlite3.Error as error:
    print("Error while working with SQLite:", error)
finally:
    # Close the database connection if it's open
    if sqliteConnection:
        sqliteConnection.close()
        # print("The SQLite connection is closed")

In [ ]:
print('Starting Short Straddle Bot')
ins = {}
for index, row in login.iterrows():
    api_key = row['apikey']
    api_secret = row['apisecret']
    symbols = eval(row['Stock'])
    kite = KiteConnect(api_key=api_key)
    print('Please Login and Access your Request Token for',row['name'],kite.login_url())
    request_token = input('Please Enter the Request Token :')
    data = kite.generate_session(request_token,api_secret=api_secret)
    access_token = data["access_token"]
    row['access_token']=access_token
    ins[row['name']]=kite


# Assuming you have imported symbols and defined the short_straddle function

def process_row(row):
    kite = ins[row['name']]
    instruments = kite.instruments()
    existing_positions = kite.positions()['net']

    for key, val in symbols.items():
        short_straddle(key[4:], val, kite, instruments, existing_positions)

# Define the time window for processing
start_time = dt_time(5, 30)  # 05:30 AM

# Use a ThreadPoolExecutor for managing concurrent processing
with ThreadPoolExecutor(max_workers=4) as executor:
    while True:
        current_time = datetime.now().time()
        if start_time != current_time:
            # Process each row concurrently
            futures = [executor.submit(process_row, row) for index, row in login.iterrows()]
            # Wait for all tasks to complete
            for future in futures:
                future.result()

            time.sleep(5)  # Adjust the delay as needed
        else:
            print('Session Ended. Please Restart')
            break  # Exit the loop when outside the processing time window

In [3]:
from datetime import datetime
import calendar

def cal_dates():
    # Calculate the dae of last friday and thursday of the current month
    year = int(datetime.now().today().strftime('%Y'))
    month = int(datetime.now().today().strftime('%m'))
    last_day = calendar.monthrange(year, month)[1]
    last_weekday = calendar.weekday(year, month, last_day)
    last_thursday = last_day - ((7 - (3 - last_weekday)) % 7)
    last_thursday_date = datetime(year, month, last_thursday).strftime('%d-%m-%Y')
    last_friday = last_day - ((7 - (4 - last_weekday)) % 7)
    last_thursday_date_dt = datetime.strptime(last_thursday_date,'%d-%m-%Y').date()
    first_weekday = calendar.weekday(year,month,1)
    days_to_add = (4-first_weekday+7)%7
    first_friday = 1 + days_to_add
    return first_friday,last_friday,last_thursday_date_dt

def long_straddle(name,val,kite,instruments,existing_positions):
    first_friday,last_friday,last_thursday_date_dt = cal_dates()
    second_last_thursday = last_friday-8
    # Check if it's time to enter the trade
    if (
        datetime.now().time() >= datetime.strptime('15:25', '%H:%M').time()
        and (
            (int(datetime.now().today().strftime('%d')) >= int(first_friday) and int(datetime.now().today().strftime('%d')) <= second_last_thursday)
        )
        ):
        if net_quant_zero(existing_positions,name):
            ltp = kite.ltp(f'NSE:{name}')[f'NSE:{name}']['last_price']
            atm = None  # Initialize ATM to None
            for i in instruments:
                if i['name'] == name:
                    if atm is None or abs(float(i['strike']) - ltp) < abs(float(atm - ltp)):
                        atm = i['strike']
            tradingsymbol_ce,lot_size_ce,tradingsymbol_pe,lot_size_pe ,instru_ce,instru_pe = get_symbol_lotsize(instruments,name,last_thursday_date_dt,atm)
            if (tradingsymbol_ce is not None and lot_size_ce is not None and tradingsymbol_pe is not None and lot_size_pe is not None):
                print(f'\nENTERING SHORT STRADDLE FOR {val} lots\n{tradingsymbol_ce} OF LOT SIZE {lot_size_ce} \nand\n{tradingsymbol_pe} of LOT SIZE {lot_size_pe}')
                # place_order(kite,tradingsymbol_ce, 0, lot_size_ce*val, kite.TRANSACTION_TYPE_SELL, KiteConnect.EXCHANGE_NFO, KiteConnect.PRODUCT_NRML,
                # KiteConnect.ORDER_TYPE_MARKET)
                # place_order(kite,tradingsymbol_pe, 0, lot_size_pe*val, kite.TRANSACTION_TYPE_SELL, KiteConnect.EXCHANGE_NFO, KiteConnect.PRODUCT_NRML,
                #             KiteConnect.ORDER_TYPE_MARKET)

    # Check if it's time to exit the trade
    if datetime.now().time() >= datetime.strptime('09:25', '%H:%M').time():
        for position in existing_positions:
            if get_name_from_instrument_token(instruments,position['instrument_token']) == name and position['quantity'] < 0 and 'CE' in position['tradingsymbol']:  # Assuming short positions
                instru_ce = position[2]
                exp,stri = get_expiry_date_and_strike_from_instrument_token(instruments,instru_ce)
                instru_pe,trad_pe = get_instru_tradesymbol_pe_from_ce(instruments,name,stri,exp)
                sell_ce = position['sell_price']
                sell_pe = get_sell_pe_from_ce(existing_positions,name)
                ltp_ce = ((kite.quote(int(instru_ce)))[str(instru_ce)])['last_price']
                ltp_pe = ((kite.quote(int(instru_pe)))[str(instru_pe)])['last_price']
                # if (
                #     (ltp_ce >= 2 * ltp_pe) or (ltp_pe >= 2 * ltp_ce)
                # or (
                #     datetime.now().today().strftime('%A') == 'Friday'  # Check if it's a Friday
                #     and datetime.now().time() >= datetime.strptime('14:00', '%H:%M').time()
                # )
                
                # or
                # (
                #     (ltp_ce <= sell_ce*0.5) or (ltp_pe <= sell_pe*0.5)
                # )
                # ):
                print(f'\nCode to Exit the Trade {name} ltp ce {ltp_ce} ,ltp pe {ltp_pe}')
                # place_order(kite,position['tradingsymbol'], 0, position['quantity'], kite.TRANSACTION_TYPE_BUY, KiteConnect.EXCHANGE_NFO, KiteConnect.PRODUCT_NRML,
                #             KiteConnect.ORDER_TYPE_MARKET)
                # place_order(kite,trad_pe, 0, position['quantity'], kite.TRANSACTION_TYPE_BUY, KiteConnect.EXCHANGE_NFO, KiteConnect.PRODUCT_NRML,
                #             KiteConnect.ORDER_TYPE_MARKET)
                # else:
                #     print(f'\n Exit Condtion not met for {name}, ltp ce {ltp_ce} ,ltp pe {ltp_pe}')
                break
